# Import

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import lxml
import sys
import time

# chromedriver 위치 설정

In [6]:
chromedriver_dir = '/Users/mongkyo/projects/mango-plate/Crawling/chromedriver'
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")
# driver = webdriver.Chrome(chormedriver_dir, chrome_option=options)

# for문을 사용하여 성수역 검색결과 1페이지 식당 들어갔다 나오기

In [7]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://www.mangoplate.com/')
all_iframes = driver.find_elements_by_tag_name("iframe")
if len(all_iframes) > 0:
    print("Ad Found\n")
    driver.execute_script("""
        var elems = document.getElementsByTagName("iframe"); 
        for(var i = 0, max = elems.length; i < max; i++)
             {
                 elems[i].hidden=true;
             }
                          """)
    print('Total Ads: ' + str(len(all_iframes)))
else:
    print('No frames found')
driver.get('https://www.mangoplate.com/')
elem = driver.find_element_by_name("main-search")
elem.send_keys("성수역")
elem.send_keys(Keys.RETURN)
# driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[1]/div[1]/figure/a').click()
# driver.execute_script("window.history.go(-1)")
# driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[1]/div[2]/figure/a').click()
# driver.execute_script("window.history.go(-1)")
# driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[2]/div[1]/figure/a').click()
# driver.execute_script("window.history.go(-1)")
# driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[2]/div[2]/figure/a').click()
# driver.execute_script("window.history.go(-1)")
# driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[3]/div[1]/figure/a').click()
# driver.execute_script("window.history.go(-1)")
# driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[3]/div[2]/figure/a').click()
# driver.execute_script("window.history.go(-1)")

for li in range(1,10+1):
    for div in range(1,2+1):
        driver.find_element_by_xpath(f'/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{li}]/div[{div}]/figure/a').click()
        
        driver.execute_script("window.history.go(-1)")



Ad Found

Total Ads: 3


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[3]/div[2]/figure/a"}
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.13.6 x86_64)


In [8]:
name = []
address = []
address_detail = []
phone_num = []
food_type = []
price_level = []
parking = []
Business_hour = []
break_time = []
last_order = []
holiday = []
website = []

In [9]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://www.mangoplate.com/')
all_iframes = driver.find_elements_by_tag_name("iframe")
if len(all_iframes) > 0:
    print("Ad Found\n")
    driver.execute_script("""
        var elems = document.getElementsByTagName("iframe"); 
        for(var i = 0, max = elems.length; i < max; i++)
             {
                 elems[i].hidden=true;
             }
                          """)
    print('Total Ads: ' + str(len(all_iframes)))
else:
    print('No frames found')
driver.get('https://www.mangoplate.com/')
elem = driver.find_element_by_name("main-search")
elem.send_keys("성수역")
elem.send_keys(Keys.RETURN)
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[1]/div[1]/figure/a').click()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
name.append(soup.select_one('body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1').get_text(strip=True))
tbody = soup.select('body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody')
for tr in tbody:
    print(tr)

Ad Found

Total Ads: 3
<tbody>
<tr class="only-desktop">
<th>주소</th>
<td>서울시 성동구 성수동2가 315-100</td>
</tr>
<tr class="only-desktop">
<th>전화번호</th>
<td>02-465-6580</td>
</tr>
<tr>
<th>음식 종류</th>
<td>
<span>탕 / 찌개 / 전골</span>
</td>
</tr>
<tr>
<th>가격대</th>
<td>만원-2만원</td>
</tr>
<tr>
<th>주차</th>
<td>무료주차 가능  </td>
</tr>
<tr>
<th style="vertical-align:top;">영업시간</th>
<td>00:00 - 24:00</td>
</tr>
<tr>
<th>메뉴</th>
<td class="menu_td">
<ul class="Restaurant_MenuList">
<li class="Restaurant_MenuItem">
<span class="Restaurant_Menu">감자국식사</span>
<span class="Restaurant_MenuPrice">7,500원</span>
</li>
<li class="Restaurant_MenuItem">
<span class="Restaurant_Menu">감자탕 (소)</span>
<span class="Restaurant_MenuPrice">24,000원</span>
</li>
<li class="Restaurant_MenuItem">
<span class="Restaurant_Menu">감자탕 (중)</span>
<span class="Restaurant_MenuPrice">27,000원</span>
</li>
<li class="Restaurant_MenuItem">
<span class="Restaurant_Menu">감자탕 (대)</span>
<span class="Restaurant_MenuPrice">36,000원</span>
</li>
</u

In [5]:
name

NameError: name 'name' is not defined

In [62]:
tmp = []

In [57]:
type(name)

str

In [60]:
tmp.append(name)

In [61]:
tmp

['소문난성수감자탕123', '소문난성수감자탕123']

In [18]:
response = requests.get(soup)
open('mango_seongsu.html', 'wt').write(response.text)
html = open('mango_seongsu.html', 'rt').read()

InvalidSchema: No connection adapters were found for '<!DOCTYPE html>
<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="성수역 맛집 인기 검색순위 | 맛집 추천, 망고플레이트" name="subject"/>
<meta content="성수역의 맛집 검색결과는 126건 입니다. 성수역의 연관 리뷰, 메뉴, 사진, 특색, 주소, 전화번호, 영업시간 등의 정보를 망고플레이트에서 확인해보세요." name="description"/>
<meta content="성수족발, 소문난감자탕, 자그마치, 그레이스톤, 깐부 치킨, 깐부 치킨, 쿤스트테이블, 명인의 제자 도제, 셰프의 국수전, 화통삼" name="keywords"/>
<meta content="https://s3-ap-northeast-1.amazonaws.com/img.prod.mp/17908/minainae__343186__img_0414_bab9bbe7.jpg" name="image"/>
<meta content="https://www.mangoplate.com/search/%EC%84%B1%EC%88%98%EC%97%AD" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="성수역 맛집 인기 검색순위 | 맛집 추천, 망고플레이트" property="og:t

In [7]:
soup = BeautifulSoup(html, 'lxml')

In [9]:
soup_ul = html.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[3]/ul')
for li in soup_ul:
    li.find_elemnet_by_xpath('/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[1]/div[1]/figure/a/div/img').click()

AttributeError: 'str' object has no attribute 'find_element_by_xpath'

In [98]:
class Restaurant:
    def __init__(self, *args, **kwargs):
        self.args = kwargs
    def __repr__(self):
        return f'{self.args}'
restaurant = Restaurant

In [99]:
restaurant(name = '성수감자탕')

{'name': '성수감자탕'}

In [100]:
dic_restaurant = {}

In [106]:
dic_restaurant = {
    1: restaurant
}

In [116]:
dic_restaurant[]

__main__.Restaurant

{}

In [117]:
category = dict()

In [ ]:
for tr in tbody:
    